<a href="https://colab.research.google.com/github/sam0786-xyz/ml-challenge/blob/main/Baseline_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import torch

In [3]:
train_path = "https://storage.googleapis.com/student-resource/student_resource/dataset/train.csv"
test_path = "https://storage.googleapis.com/student-resource/student_resource/dataset/test.csv"

In [5]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   sample_id        75000 non-null  int64  
 1   catalog_content  75000 non-null  object 
 2   image_link       75000 non-null  object 
 3   price            75000 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 2.3+ MB


In [ ]:
train_df.isnull().sum()

,0
sample_id,0
catalog_content,0
image_link,0
price,0


In [ ]:
train_df.duplicated().sum()

np.int64(0)

In [7]:
import re
def clean_text(txt):
    txt = str(txt).lower()
    txt = re.sub(r'item name:|bullet point \d+:|product description:', ' ', txt)
    txt = re.sub(r'[^a-z0-9 ]', ' ', txt)
    txt = re.sub(r'\s+', ' ', txt).strip()
    return txt

train_df['catalog_content'] = train_df['catalog_content'].fillna('').apply(clean_text)
test_df['catalog_content'] = test_df['catalog_content'].fillna('').apply(clean_text)

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
show_progress_bar = True
train_text_emb = model.encode(
    train_df['catalog_content'].tolist(),
    device=device,
    show_progress_bar=True,
    batch_size=64   # optional but recommended
)

print("Shape of the embedding matrix:", train_text_emb.shape)


Batches:   0%|          | 0/1172 [00:00<?, ?it/s]

Shape of the embedding matrix: (75000, 384)


In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
show_progress_bar = True
test_text_emb = model.encode(test_df['catalog_content'].tolist(), batch_size=64)

print("Shape of the embedding matrix:", test_text_emb.shape)

Shape of the embedding matrix: (75000, 384)


In [12]:
from sentence_transformers import SentenceTransformer
img_model = SentenceTransformer('clip-ViT-B-32')

train_img_emb = img_model.encode(train_df['image_link'].tolist(), show_progress_bar=True)
test_img_emb = img_model.encode(test_df['image_link'].tolist(), batch_size=16, show_progress_bar=True)

modules.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

0_CLIPModel/pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

0_CLIPModel/model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Batches:   0%|          | 0/2344 [00:00<?, ?it/s]

Batches:   0%|          | 0/4688 [00:00<?, ?it/s]

In [13]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = np.hstack([train_text_emb, train_img_emb])
X_test = np.hstack([test_text_emb, test_img_emb])
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y = train_df['price'].values

In [18]:
from sklearn.model_selection import train_test_split

X_train_split, X_val, y_train, y_val = train_test_split(
    X_train, y, test_size=0.2, random_state=42
)


In [14]:
import numpy as np
price_q1, price_q3 = np.percentile(y, [5, 95])
mask = (y >= price_q1) & (y <= price_q3)
X_train_filtered, y_filtered = X_train[mask], y[mask]

In [23]:
!pip install -U lightgbm


In [25]:
from lightgbm import LGBMRegressor, early_stopping, log_evaluation

lgb = LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.02,
    num_leaves=128,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=0.3,
    device='gpu'
)

lgb.fit(
    X_train_split, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='mae',
    callbacks=[
        early_stopping(stopping_rounds=50),
        log_evaluation(period=100)
    ]
)


[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 228480
[LightGBM] [Info] Number of data points in the train set: 60000, number of used features: 896
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 896 dense feature groups (51.27 MB) transferred to GPU in 0.100083 secs. 0 sparse feature groups
[LightGBM] [Info] Start training from score 23.598634
Training until validation scores don't improve for 50 rounds
[100]	valid_0's l1: 16.7177	valid_0's l2: 1321.04
[200]	valid_0's l1: 16.155	valid_0's l2: 1271.32
[300]	valid_0's l1: 15.9181	valid_0's l2: 1248.48
[400]	valid_0's l1: 15.7929	valid_0's l2: 1238.56
[500]	valid_0's l1: 15.7073	valid_0's l2: 1230.44
[600]	valid_0's l1: 15.6479	valid_0's l2: 1224.68
[700]	valid_0's l1: 15.6052	valid_0's l2: 1221.13
[800]	valid_0's l1:

LGBMRegressor(colsample_bytree=0.8, device='gpu', learning_rate=0.02,
              n_estimators=2000, num_leaves=128, reg_alpha=0.1, reg_lambda=0.3,
              subsample=0.8)

In [26]:
def smape(y_true, y_pred):
    return 100 * np.mean(
        2 * np.abs(y_pred - y_true) /
        (np.abs(y_true) + np.abs(y_pred))
    )

val_pred = lgb.predict(X_val)
print(f"Validation SMAPE: {smape(y_val, val_pred):.2f}%")


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


Validation SMAPE: 68.17%


In [ ]:
import numpy as np

def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# After training
val_pred = lgb.predict(X_val)
print(f"Validation SMAPE: {smape(y_val, val_pred):.2f}%")


In [32]:
submission = pd.DataFrame({
    "sample_id": test_df["sample_id"],
    "price": lgb.predict(X_test)  # Generate predictions for the entire test set
})
submission.to_csv("test_out.csv", index=False)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
